# Migrate from production to staging and viceversa using LMIPy

Here is some sample text, and a first block of code

In [ ]:
import LMIPy
import numpy as np
import requests
import json
import getpass
import os
from pprint import pprint
print(LMIPy.__version__)


In [ ]:
def login(email, password, env = 'production'):
    urlEnvirionments = {
        'production': f'https://api.resourcewatch.org/auth/login'
        'staging': f'https://staging-api.resourcewatch.org/auth/login'
    }
    headers = {'Content-Type': 'application/json'}
    payload = {
    "email": f"{email}",
    "password": f"{password}"
    }
    r = requests.post(url, data=json.dumps(payload), headers=headers)

    return r.json().get('data').get('token')

## Login into the rw production Api

In [ ]:
email = 'alicia.arenzana@vizzuality.com'
password = getpass.getpass('Login password:')

In [ ]:
tokenProduction = login(email, password)

## Login into the rw staging Api

In [ ]:
email = 'alicia.arenzana@vizzuality.com'
password = getpass.getpass('Login password:')

In [ ]:
tokenStaging = login(email, password, env = 'staging')

## Using LMIPy

### Check `aqueduct-water-risk` datasets

In [ ]:
prod_server = "https://api.resourcewatch.org"
app = ["rw"]
env = 'production'

collection = LMIPy.Collection(app=app, env=env, server=prod_server, limit=1000)

In [ ]:
collection[0]

### Creating a local backup of `aqueduct-water-risk` production Data objects

In [ ]:
path = 'RW-PROD-BACKUP'

collection.save(path)

**Load data objects from local backup**

In [ ]:
files = os.listdir(path)

with open(path+'/'+files[0]) as json_file:
    data = json.load(json_file)
    
pprint(data)

### Delete production Data objects

**Delete layers**

In [ ]:
prod_server = "https://api.resourcewatch.org"
app = ["aqueduct-water-risk"]
env = 'production'
object_type = ['layer']

collection = LMIPy.Collection(app=app, env=env, object_type=object_type, server=prod_server, limit=1000)

for n in range(len(collection)):
    ly = LMIPy.Layer(id_hash=collection[n].id)
    ly.delete(token=token, force=True)

**Delete datasets**

In [ ]:
prod_server = "https://api.resourcewatch.org"
app = ["aqueduct-water-risk"]
env = 'production'
object_type = ['dataset']

collection = LMIPy.Collection(app=app, env=env, object_type=object_type, server=prod_server, limit=1000)

for n in range(len(collection)):
    ds = LMIPy.Dataset(id_hash=collection[n].id)
    ds.delete(token=token, force=True)

### Clone staging Data objects into production

**Clone Water Risk datasets abd layers**

In [ ]:
staging_server = "https://staging-api.globalforestwatch.org"
prod_server = "https://api.resourcewatch.org"
env = 'production'

In [ ]:
datasets_ids = [
]

datasets = []
for id in datasets_ids:
    ds = LMIPy.Dataset(id, server=staging_server)
    print(ds)
    datasets.append(ds)

In [ ]:
for dataset in datasets:
    dataset.clone(token=token, env=env, clone_server=prod_server, dataset_params=dataset.attributes, clone_children=True)